In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset, Dataset, concatenate_datasets
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

In [2]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             #load_in_8bit=True,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto"
                                            )

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [3]:
import pandas as pd

data_train_tw = pd.read_csv('academic_paper_2015-2024.csv', encoding = 'utf-8-sig')
print(data_train_tw)

                                                Authors  \
0             Samadzad M.; Ansari F.; Afshari Moez M.A.   
1                      Kim S.-W.; Kwon D.-H.; Cho I.-H.   
2                    Qin V.L.; Ding G.; Balakrishnan H.   
3                                    Sah B.; Titiyal R.   
4     Di Mascio P.; Celesti M.; Sabatini M.; Moretti L.   
...                                                 ...   
2543                                            Do K.D.   
2544      Sinha P.; Stoll A.M.; Stilson E.V.; Bevirt J.   
2545               Güçlü A.; Arıkan K.B.; Kurtuluş D.F.   
2546        Oberschwendtner S.; Roessler C.; Hornung M.   
2547  Yang Y.; Karimadini M.; Xiang C.; Teo S.H.; Ch...   

                                      Author full names  \
0     Samadzad, Mahdi (50861885300); Ansari, Fatemeh...   
1     Kim, Seon-Woong (59347621400); Kwon, Do-Hun (5...   
2     Qin, Victor L. (57446368000); Ding, Geoffrey (...   
3     Sah, Bhawesh (57195055488); Titiyal, Rohit (57...

In [4]:
data_train_tw['cleaned_text'] = data_train_tw['Title'] + data_train_tw['Abstract']

In [6]:
def generate_prompt(content):
    begin = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>"
    #syst = "<<SYS>> You are a helpful AI assistant that answers questions briefly and directly.\n If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>\n"
    #inst = "Read the following text. Does it mention the Gilbert damping constant of a certain material? If so, list the corresponding material and its Gilbert damping canstant.\n" + content
    syst = "You are a helpful AI assistant that answers questions briefly and directly.<|eot_id|>\n<|start_header_id|>user<|end_header_id|>"
    inst = "The following texts are research or tweets about urban low-altitude transportation. Rank the attitude as positive, negative, or neutral, and categorize the discussion content into one of the following categories: \n1: Safety \n2: Aerodynamics \n3: Integration and Infrastructure \n4: Automation \n5: Price and cost \n6: Policy \n7: User experience \nOnly answer the attitude and category number, don't answer anything else. For example: Positive, 5\nText: "+content
    end = "<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n"
    prompt = (" ").join([begin, syst, inst, end])
    return prompt

#print(generate_prompt('How are you?'))
#print(generate_prompt(data_train['text_cleaned'][1]))
print(generate_prompt(data_train_tw['cleaned_text'][1]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a helpful AI assistant that answers questions briefly and directly.<|eot_id|>
<|start_header_id|>user<|end_header_id|> The following texts are research or tweets about urban low-altitude transportation. Rank the attitude as positive, negative, or neutral, and categorize the discussion content into one of the following categories: 
1: Safety 
2: Aerodynamics 
3: Integration and Infrastructure 
4: Automation 
5: Price and cost 
6: Policy 
7: User experience 
Only answer the attitude and category number, don't answer anything else. For example: Positive, 5
Text: Temperature Management Strategy for Urban Air Mobility Batteries to Improve Energy Efficiency in Low-Temperature ConditionsAs urban population concentration accelerates, issues such as traffic congestion caused by automobiles and climate change due to carbon dioxide emissions are becoming increasingly severe. Recently, urban air mobility (UAM) has been attracting 

In [ ]:
import pandas as pd
import numpy as np
import time

data = []
save_interval = 20

start_time = time.time()  # 记录开始时间

for i in range(len(data_train_tw)):#['text_cleaned']):
    if i % 1 == 0:
        #if pd.isna(data_train_tw.iloc[i]['result']) and not pd.isna(data_train_tw.iloc[i]['cleaned_text']):
        if not pd.isna(data_train_tw.iloc[i]['cleaned_text']):
            #print(i)
            
            input_prompt = generate_prompt(data_train_tw.iloc[i]['cleaned_text'])
            #input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
            #inputs = tokenizer(input_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
            inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True).to("cuda")
            input_tokens = inputs["input_ids"]
            attention_mask = inputs["attention_mask"]
            with torch.cuda.amp.autocast():
                generation_output = model.generate(
                    input_ids=input_tokens,
                    attention_mask=attention_mask,
                    max_new_tokens=8,
                    do_sample=False,
                    repetition_penalty=1.1,
                    num_return_sequences=1,
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.eos_token_id
                )
            op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
            #print(op)

            inst_index = op.find('assistant\n')
        
            if inst_index != -1:
                #print(text)
                #print(op[inst_index + len('assistant\n'):])
                #data.append({"number": i, "link": data_train_tw.iloc[i]['link'], "text": data_train_tw.iloc[i]['cleaned_text'], "result": op[inst_index + len('assistant\n'):].replace('\n', ' ')})
                data.append({"number": i, "link": data_train_tw.iloc[i]['Title'], "text": data_train_tw.iloc[i]['Abstract'], "result": op[inst_index + len('assistant\n'):].replace('\n', ' ')})
            else:
                print("未找到'assistant\n'标记")
                #data.append({"number": i, "text": text, "sentiment":""})
    
            # 每 save_interval 个迭代保存一次
            if len(data) == save_interval:
                df = pd.DataFrame(data)
                df.to_csv("output_tw.csv", encoding = 'utf-8-sig', index=False, mode='a', header=False)  # 追加模式
                data = []
                end_time = time.time()  # 记录结束时间
                elapsed_time = end_time - start_time  # 计算用时
                print(f"已保存到 output.csv，当前迭代次数：{i}，用时：{elapsed_time:.2f} 秒")
        
                start_time = end_time  # 更新开始时间，用于计算下一个周期的用时

# 最后一次保存
df = pd.DataFrame(data)
df.to_csv("output_tw.csv", encoding = 'utf-8-sig', index=False, mode='a', header=False)
print("已保存到 output_tw.csv")
            

已保存到 output.csv，当前迭代次数：19，用时：9.96 秒
已保存到 output.csv，当前迭代次数：39，用时：9.99 秒
已保存到 output.csv，当前迭代次数：59，用时：9.87 秒
已保存到 output.csv，当前迭代次数：79，用时：9.92 秒
已保存到 output.csv，当前迭代次数：99，用时：9.94 秒
已保存到 output.csv，当前迭代次数：119，用时：9.66 秒
已保存到 output.csv，当前迭代次数：139，用时：9.98 秒
已保存到 output.csv，当前迭代次数：159，用时：9.94 秒
已保存到 output.csv，当前迭代次数：179，用时：9.83 秒
已保存到 output.csv，当前迭代次数：199，用时：9.96 秒
已保存到 output.csv，当前迭代次数：219，用时：9.88 秒
已保存到 output.csv，当前迭代次数：239，用时：9.86 秒
已保存到 output.csv，当前迭代次数：259，用时：9.80 秒
已保存到 output.csv，当前迭代次数：279，用时：9.93 秒
已保存到 output.csv，当前迭代次数：299，用时：10.23 秒
已保存到 output.csv，当前迭代次数：319，用时：9.97 秒
已保存到 output.csv，当前迭代次数：339，用时：9.96 秒
已保存到 output.csv，当前迭代次数：359，用时：9.95 秒
已保存到 output.csv，当前迭代次数：379，用时：9.87 秒
已保存到 output.csv，当前迭代次数：399，用时：9.85 秒
已保存到 output.csv，当前迭代次数：419，用时：10.08 秒
已保存到 output.csv，当前迭代次数：439，用时：9.92 秒


In [4]:
data = load_dataset("enyuan/Abstracts")
data_train = data["train"]

custom_data = load_dataset('json', data_files='data_eval.json')
data_val = custom_data['train']

# Print the dataset details
print(data_train)
print(data_val)

# Access an example
#example = data_train[0]
#print(example)

def generate_prompt(title, abstract=None, eos_token="</s>"):
  instruction = "The abstract of the paper:\n"
  input = f"{title}\n"
  abstract = f"Abstract: {abstract + ' ' + eos_token if abstract else ''} "
  prompt = (" ").join([instruction, input, abstract])
  return prompt

print(generate_prompt(data_train[0]["title"], data_train[0]["abstract"]))

Dataset({
    features: ['title', 'doi', 'abstract', 'publicationDate'],
    num_rows: 165071
})
Dataset({
    features: ['title', 'abstract', 'publicationDate'],
    num_rows: 559
})
The abstract of the paper:
 Inconel 625 sustainable milling surface integrity and the dependence on alloy processing route
 Abstract: The discovery of deepwater oil and gas sources has altered the scenario of world production of oil products, attracting even more attention to nickel superalloys. However, this class of materials can be used in several applications. Furthermore, nickel superalloys are highly dependent on their processing history, and the manner in which superalloys react to machining can directly affect the finished product. This work aims to evaluate the surface integrity of two different materials after cryogenic side-milling in conditions that stimulate severe plastic deformation (SPD) and high heat generation. The results show that the material response to machining depends strongly on 

In [4]:
input_prompt = generate_prompt(data_train[50]["title"])
input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
with torch.cuda.amp.autocast():
  generation_output = model.generate(
      input_ids=input_tokens,
      max_new_tokens=1000,
      do_sample=True,
      top_k=10,
      top_p=0.9,
      temperature=0.3,
      repetition_penalty=1.15,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
    )
op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(op)

The abstract of the paper:
 Effect of cryogenic cooling on residual stresses and surface finish of 316L during hybrid manufacturing
 Abstract:   In this work, a novel approach for reducing the residual stress in the welded joints of stainless steel is presented. A new process called Hybrid Manufacturing (HM) was developed to reduce the residual stress in the welded joints by using two different techniques namely, laser beam welding (LBW) and cryogenic treatment (CT). The effectiveness of HM technique has been studied with respect to the reduction of residual stress and improvement in surface roughness. The results showed that the residual stress can be reduced up to 40% when compared to conventional LBW method. Moreover, the surface roughness can also be improved significantly as shown by the Ra value which decreases from 25.87µm to 19.31µm after CT.
The full text of the article: http://www.sciencedirect.com/science/article/pii/S092583881400132X


In [4]:
with open('materials.txt', 'r') as file:
    word_list = file.read().splitlines()